<a href="https://colab.research.google.com/github/levynlee/ESAA/blob/main/1004_%ED%95%84%EC%82%AC%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**7. 문서 군집화 소개와 실습(Opinion Review 데이터 세트)**

###**7.1 문서 군집화 개념**

**문서 군집화**는 비슷한 텍스트 구성의 문서를 군집화하는 것이다. 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있으므로 텍스트 분류 기반의 문서 분류와 유사하다. 하지만 텍스트 분류 기반의 문서 분류는 사전에 결정 카테고리 값을 가진 학습 데이터 세트가 필요한 데 반해, 문서 군집화는 학습 데이터 세트가 필요 없는 **비지도학습** 기반으로 동작한다.

###**7.2 Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기**

In [2]:
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 700)

path = r'/content/topics'
# path로 저장한 디렉터리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합
# 개별 파일들의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')
    # 절대경로로 주어진 파일명을 가공
    # 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]
    #파일명 리스트와 파일 내용 리스트에 파일명과 파일 내용을 추가
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일 내용 리스트를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,/content/topics/directions_garmin_nuvi_255W_gps,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...
1,/content/topics/display_garmin_nuvi_255W_gps,"3 quot widescreen display was a bonus .\n0 This made for smoother graphics on the 255w of the vehicle moving along displayed roads, where the 750's display was more of a jerky movement .\n1 ..."
2,/content/topics/satellite_garmin_nuvi_255W_gps,"It's fast to acquire satellites .\n0 If you've ever had a Brand X GPS take you on some strange route that adds 20 minutes to your trip, has you turn the wrong way down a one way road, tell you to turn AFTER you've passed the street, frequently loses the satellite signal, or has old maps missing streets, you know how important this stuff is .\n1 ..."
3,/content/topics/screen_netbook_1005ha,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...
4,/content/topics/screen_ipod_nano_8gb,"As always, the video screen is sharp and bright .\n0 2, inch screen and a glossy, polished aluminum finish that one CNET editor described as looking like a Christmas tree ornament .\n1 ..."


각 파일 이름 자체만으로 의견(opinion)의 텍스트가 어떠한 제품/서비스에 대한 리뷰인지 잘 알 수 있다.  

문서를 TF-IDF 형태로 피처 벡터화하자. TfidfVectorizer는 Lemmatization 같은 어근 변환을 직접 지원하진 않지만 tokenizer 인자에 커스텀 어근 변환 함수를 적용해 어근 변환을 수행할 수 있다. 이를 위해 LemNormalize() 함수를 만든다. ngram은 (1,2)로 하고, min_df와 max_df 범위를 설정해 피처의 개수를 제한한다. TfidfVectorizer와 fit_transform()의 인자로 document_df DataFrame의 opinion_text 칼럼을 입력하면 개별 문서 텍스트에 대해 TF-IDF 변환된 피처 벡터화된 행렬을 구할 수 있다.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet') # Download required resource

# LemNormalize() 함수 정의
def LemNormalize(text):
    """
    Lemmatizes and normalizes text
    """
    wordnet_lemmatizer = WordNetLemmatizer()
    return [wordnet_lemmatizer.lemmatize(w) for w in text.split()]

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 칼럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

[nltk_data] Downloading package wordnet to /root/nltk_data...


문서별 텍스트가 TF-IDF 변환된 피처 벡터화 행렬에 대해서 군집화를 수행해 어떤 문서끼리 군집되는지 확인해보자. 군집화 기법은 K-평균을 적용한다. 문서의 유형은 크게 보자면 전자제품, 자동차, 호텔로 되어 있다. 전자 제품은 다시 내비게이션, 아이팟, 킨들, 랩톱 컴퓨터 등과 같은 세부 요소로 나뉜다. 먼저 5개의 중심(Centroid) 기반으로 어떻게 군집되는지 확인해본다. 최대 반복 횟수 max_iter는 10000으로 설정한다. KMeans를 수행한 후에 군집의 Label 값과 중심별로 할당된 데이터 세트의 좌표 값을 구한다.

In [5]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

각 데이터별로 할당된 군집의 레이블을 파일명과 파일 내용을 가지고 있는 document_df DataFrame에 'cluster_label' 칼럼을 추가해 저장한다. 각 파일명은 의견 리뷰에 대한 주제를 나타낸다. 군집이 각 주제별로 유사한 형태로 잘 구성됐는지 알아본다.

In [6]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,/content/topics/directions_garmin_nuvi_255W_gps,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...,2
1,/content/topics/display_garmin_nuvi_255W_gps,"3 quot widescreen display was a bonus .\n0 This made for smoother graphics on the 255w of the vehicle moving along displayed roads, where the 750's display was more of a jerky movement .\n1 ...",2
2,/content/topics/satellite_garmin_nuvi_255W_gps,"It's fast to acquire satellites .\n0 If you've ever had a Brand X GPS take you on some strange route that adds 20 minutes to your trip, has you turn the wrong way down a one way road, tell you to turn AFTER you've passed the street, frequently loses the satellite signal, or has old maps missing streets, you know how important this stuff is .\n1 ...",2
3,/content/topics/screen_netbook_1005ha,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...,1
4,/content/topics/screen_ipod_nano_8gb,"As always, the video screen is sharp and bright .\n0 2, inch screen and a glossy, polished aluminum finish that one CNET editor described as looking like a Christmas tree ornament .\n1 ...",1


In [9]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
38,/content/topics/comfort_honda_accord_2008,"Drivers seat not comfortable, the car itself compared to other models of similar class .\n0 ...",0
31,/content/topics/comfort_toyota_camry_2007,"Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 Seats are fine, in fact of all the smaller sedans this is the most comfortable I found for the price as I am 6', 2 and 250# .\n1 Great gas mileage and comfortable on long trips ...",0
26,/content/topics/gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 ...,0
48,/content/topics/interior_honda_accord_2008,I love the new body style and the interior is a simple pleasure except for the center dash .\n0 ...,0
27,/content/topics/interior_toyota_camry_2007,"First of all, the interior has way too many cheap plastic parts like the cheap plastic center piece that houses the clock .\n0 3 blown struts at 30,000 miles, interior trim coming loose and rattling squeaking, stains on paint, and bug splats taking paint off, premature uneven brake wear, on 3rd windsh...",0
32,/content/topics/mileage_honda_accord_2008,"It's quiet, get good gas mileage and looks clean inside and out .\n0 The mileage is great, and I've had to get used to stopping less for gas .\n1 Thought gas ...",0
47,/content/topics/performance_honda_accord_2008,"Very happy with my 08 Accord, performance is quite adequate it has nice looks and is a great long, distance cruiser .\n0 6, 4, 3 eco engine has poor performance and gas mileage of 22 highway .\n1 Overall performance is good but comfort level is poor .\n2 ...",0
8,/content/topics/quality_toyota_camry_2007,I previously owned a Toyota 4Runner which had incredible build quality and reliability .\n0 I bought the Camry because of Toyota reliability and qua...,0
40,/content/topics/seats_honda_accord_2008,"Front seats are very uncomfortable .\n0 No memory seats, no trip computer, can only display outside temp with trip odometer .\n1 ...",0
10,/content/topics/transmission_toyota_camry_2007,"After slowing down, transmission has to be kicked to speed up .\n0 ...",0


Cluster #0은 호텔에 대한 리뷰로 군집화되었다.

In [10]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
16,/content/topics/battery-life_amazon_kindle,"After I plugged it in to my USB hub on my computer to charge the battery the charging cord design is very clever !\n0 After you have paged tru a 500, page book one, page, at, a, time to get from Chapter 2 to Chapter 15, see how excited you are about a low battery and all the time it took to get there !\n1 ...",1
30,/content/topics/battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\n0 I love this ipod except for the battery life .\n1 ...,1
17,/content/topics/battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery , and a 1 .\n0 Not to mention that as of now...",1
45,/content/topics/performance_netbook_1005ha,"The Eee Super Hybrid Engine utility lets users overclock or underclock their Eee PC's to boost performance or provide better battery life depending on their immediate requirements .\n0 In Super Performance mode CPU, Z shows the bus speed to increase up to 169 .\n1 One...",1
4,/content/topics/screen_ipod_nano_8gb,"As always, the video screen is sharp and bright .\n0 2, inch screen and a glossy, polished aluminum finish that one CNET editor described as looking like a Christmas tree ornament .\n1 ...",1
3,/content/topics/screen_netbook_1005ha,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...,1
5,/content/topics/size_asus_netbook_1005ha,"A few other things I'd like to point out is that you must push the micro, sized right angle end of the ac adapter until it snaps in place or the battery may not charge .\n0 The full size right shift k...",1
36,/content/topics/sound_ipod_nano_8gb,headphone jack i got a clear case for it and it i got a clear case for it and it like prvents me from being able to put the jack all the way in so the sound can b messsed up or i can get it in there and its playing well them go to move or something and it slides out .\n0 Picture and sound quality are excellent for this typ of devic .\n1 ...,1
44,/content/topics/video_ipod_nano_8gb,"I bought the 8, gig Ipod Nano that has the built, in video camera .\n0 Itunes has an on, line store, where you may purchase and download music and videos which will install onto the ipod .\n1 ...",1


Cluster #1은 킨들, 아이팟, 넷북 등의 포터블 전자기기 및 주요 구성요소(배터리, 키보드 등)에 대한 리뷰로 군집화되었다.

In [11]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
20,/content/topics/accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1 This functi...",2
19,/content/topics/buttons_amazon_kindle,"I thought it would be fitting to christen my Kindle with the Stephen King novella UR, so went to the Amazon site on my computer and clicked on the button to buy it .\n0 As soon as I'd clicked the button to confirm my order it appeared on my Kindle almost immediately !\n1 ...",2
0,/content/topics/directions_garmin_nuvi_255W_gps,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...,2
1,/content/topics/display_garmin_nuvi_255W_gps,"3 quot widescreen display was a bonus .\n0 This made for smoother graphics on the 255w of the vehicle moving along displayed roads, where the 750's display was more of a jerky movement .\n1 ...",2
14,/content/topics/eyesight-issues_amazon_kindle,"It feels as easy to read as the K1 but doesn't seem any crisper to my eyes .\n0 the white is really GREY, and to avoid considerable eye, strain I had to refresh pages every other page .\n1 The dream has always been a portable electronic device that could hold a ton of reading material, automate subscriptions and fa...",2
28,/content/topics/features_windows7,"I had to uninstall anti, virus and selected other programs, some of which did not have listings in the Programs and Features Control Panel section .\n0 This review briefly touches upon some of the key features and enhancements of Microsoft's latest OS .\n1 ...",2
15,/content/topics/fonts_amazon_kindle,"Being able to change the font sizes is awesome !\n0 For whatever reason, Amazon decided to make the Font on the Home Screen ...",2
50,/content/topics/keyboard_netbook_1005ha,", I think the new keyboard rivals the great hp mini keyboards .\n0 Since the battery life difference is minimum, the only reason to upgrade would be to get the better keyboard .\n1 The keyboard is now as good as t...",2
33,/content/topics/navigation_amazon_kindle,"In fact, the entire navigation structure has been completely revised , I'm still getting used to it but it's a huge step forward .\n0 ...",2
2,/content/topics/satellite_garmin_nuvi_255W_gps,"It's fast to acquire satellites .\n0 If you've ever had a Brand X GPS take you on some strange route that adds 20 minutes to your trip, has you turn the wrong way down a one way road, tell you to turn AFTER you've passed the street, frequently loses the satellite signal, or has old maps missing streets, you know how important this stuff is .\n1 ...",2


Cluster #2는 Cluster #1과 비슷하게 킨들 등이 군집에 포함되어 있지만, 주로 차량용내비게잉션으로 군집이 구성되었다. (파일명 xxx_garmin_nuvi_xxx는 차량용 내비게이션 리뷰이다.)

In [12]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
42,/content/topics/food_holiday_inn_london,The room was packed to capacity with queues at the food buffets .\n0 The over zealous staff cleared our unfinished drinks while we were collecting cooked food and movement around the room with plates was difficult in the crowded circumstances .\n1 ...,3
34,/content/topics/food_swissotel_chicago,The food for our event was delicious .\n0 ...,3
13,/content/topics/location_bestwestern_hotel_sfo,"Good Value good location , ideal choice .\n0 Great Location , Nice Rooms , Helpless Concierge\n1 ...",3
11,/content/topics/location_holiday_inn_london,"Great location for tube and we crammed in a fair amount of sightseeing in a short time .\n0 All in all, a normal chain hotel on a nice lo...",3
41,/content/topics/parking_bestwestern_hotel_sfo,Parking was expensive but I think this is common for San Fran .\n0 there is a fee for parking but well worth it seeing no where to park if you do have a car .\n1 ...,3
29,/content/topics/price_amazon_kindle,"If a case was included, as with the Kindle 1, that would have been reflected in a higher price .\n0 lower overall price, with nice leather cover .\n1 ...",3
49,/content/topics/price_holiday_inn_london,"All in all, a normal chain hotel on a nice location , I will be back if I do not find anthing closer to Picadilly for a better price .\n0 ...",3


Cluster #3은 킨들 리뷰가 한 개 섞여 있는 것이 살짝 아쉽지만, Cluster #0과 같이 대부분 호텔에 대한 리뷰로 군집화되어 있다.

In [13]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
39,/content/topics/bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0 The second room was smaller, with a very inconvenient bathroom layout, but at least it was quieter and we were able to sleep .\n1 ...",4
43,/content/topics/free_bestwestern_hotel_sfo,The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room .\n0 They also have a computer available with free internet which is a nice bonus but I didn't find that out till the day before we left but was still able to get on there to check our flight to Vegas the next day .\n1 ...,4
23,/content/topics/room_holiday_inn_london,"We arrived at 23,30 hours and they could not recommend a restaurant so we decided to go to Tesco, with very limited choices but when you are hingry you do not careNext day they rang the bell at 8,00 hours to clean the room, not being very nice being waken up so earlyEvery day they gave u...",4
12,/content/topics/rooms_bestwestern_hotel_sfo,"Great Location , Nice Rooms , H...",4
25,/content/topics/rooms_swissotel_chicago,"The Swissotel is one of our favorite hotels in Chicago and the corner rooms have the most fantastic views in the city .\n0 The rooms look like they were just remodled and upgraded, there was an HD TV and a nice iHome docking station to put my iPod so I could set the alarm to wake up with my music instead of the radio .\n1 ...",4
18,/content/topics/service_bestwestern_hotel_sfo,"Both of us having worked in tourism for over 14 years were very disappointed at the level of service provided by this gentleman .\n0 The service was good, very friendly staff and we loved the free wine reception each night .\n1 ...",4
22,/content/topics/service_holiday_inn_london,"not customer, oriented hotelvery low service levelboor reception\n0 The room was quiet, clean, the bed and pillows were comfortable, and the serv...",4
46,/content/topics/service_swissotel_hotel_chicago,Mediocre room and service for a very extravagant price .\n0 ...,4
37,/content/topics/staff_bestwestern_hotel_sfo,Staff are friendl...,4
9,/content/topics/staff_swissotel_chicago,"The staff at Swissotel were not particularly nice .\n0 Each time I waited at the counter for staff for several minutes and then was waved to the desk upon my turn with no hello or anything, or apology for waiting in line .\n1 ...",4


Cluster #4는 토요타와 혼다 등의 자동차에 대한 리뷰로 잘 군집화되었다.

전반적으로 군집화된 결과를 살펴보면 군집 개수가 약간 많게 설정되어 있어 세분화되어 군집화된 경향이 있다. 중심 개수를 5개에서 3개로 낮춰서 3개 그룹으로 군집화한 뒤 결과를 확인해본다.

In [14]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

# 소속 클러스터를 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
25,/content/topics/rooms_swissotel_chicago,"The Swissotel is one of our favorite hotels in Chicago and the corner rooms have the most fantastic views in the city .\n0 The rooms look like they were just remodled and upgraded, there was an HD TV and a nice iHome docking station to put my iPod so I could set the alarm to wake up with my music instead of the radio .\n1 ...",0
40,/content/topics/seats_honda_accord_2008,"Front seats are very uncomfortable .\n0 No memory seats, no trip computer, can only display outside temp with trip odometer .\n1 ...",0
41,/content/topics/parking_bestwestern_hotel_sfo,Parking was expensive but I think this is common for San Fran .\n0 there is a fee for parking but well worth it seeing no where to park if you do have a car .\n1 ...,0
39,/content/topics/bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0 The second room was smaller, with a very inconvenient bathroom layout, but at least it was quieter and we were able to sleep .\n1 ...",0
18,/content/topics/service_bestwestern_hotel_sfo,"Both of us having worked in tourism for over 14 years were very disappointed at the level of service provided by this gentleman .\n0 The service was good, very friendly staff and we loved the free wine reception each night .\n1 ...",0
49,/content/topics/price_holiday_inn_london,"All in all, a normal chain hotel on a nice location , I will be back if I do not find anthing closer to Picadilly for a better price .\n0 ...",0
26,/content/topics/gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 ...,0
27,/content/topics/interior_toyota_camry_2007,"First of all, the interior has way too many cheap plastic parts like the cheap plastic center piece that houses the clock .\n0 3 blown struts at 30,000 miles, interior trim coming loose and rattling squeaking, stains on paint, and bug splats taking paint off, premature uneven brake wear, on 3rd windsh...",0
38,/content/topics/comfort_honda_accord_2008,"Drivers seat not comfortable, the car itself compared to other models of similar class .\n0 ...",0
13,/content/topics/location_bestwestern_hotel_sfo,"Good Value good location , ideal choice .\n0 Great Location , Nice Rooms , Helpless Concierge\n1 ...",0


앞에서처럼 군집별로 분류된 것들을 확인해보면, Cluster #0은 포터블 전자기기 리뷰로, Cluster #1은 자동차 리뷰로, Cluster #2는 호텔 리뷰로 잘 구성되었다.

###**7.3 군집별 핵심 단어 추출하기**

각 군집에 속한 문서는 핵심 단어를 주축으로 군집화되어 있을 것이다. 이번에는 각 군집을 구성하는 핵심 단어가 어떤 것이 있는지 확인해본다.  

KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심을 기준으로 얼마나 가깝게 위치해 있는지 cluster_centers_라는 속성으로 제공한다. cluster_centers_는 배열 값으로 제공되며, 행은 개별 군집을, 열은 개별 피처를 의미한다. 각 배열 내의 값은 개별 군집 내의 상대 위치를 숫자 값으로 표현한 일종의 좌표 값이다. 예를 들어 cluster_centers[0,1]은 0번 군집에서 두 번째 피처의 위치 값이다.

In [15]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape : ', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape :  (3, 5649)
[[0.         0.00030572 0.00069024 ... 0.0008395  0.00055955 0.00055955]
 [0.0014943  0.0014943  0.         ... 0.         0.         0.        ]
 [0.00154391 0.00095784 0.         ... 0.00064271 0.00064271 0.00064271]]


각 배열값은 0에서 1까지의 값을 가질 수 있으며 1에 가까울수록 중심과 가까운 값을 의미한다.  

이제 cluster_centers_ 속성값을 이용해 각 군집별 핵심 단어를 찾는다. cluster_centers_ 속성은 넘파이의 ndarray이다. ndarray의 argsort()[:,::-1]를 이용하면 cluster_centers 배열 내 값이 큰 순으로 정렬된 위치 인덱스 값을 반환한다. 이 위치 인덱스 값이 필요한 이유는 핵심 단어 피처의 이름을 출력하기 위해서이다. 새로운 함수 get_cluster_details()를 생성해 위에 대한 처리를 맡긴다. cluster_centers_ 배열 내에서 가장 값이 큰 데이터의 위치 인덱스를 추출한 뒤, 해당 인덱스를 이용해 핵심 단어 이름과 그때의 상대 위치 값을 추출해 cluster_details라는 Dict 객체 변수에 기록하고 반환하는 것이 get_cluster_details() 함수의 주요 로직이다.

In [24]:
# 각 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 반환
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
  cluster_details = {}

  # cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
  centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, ::-1]

  # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num] = {}
    cluster_details[cluster_num]['cluster'] = cluster_num

    # cluster_centers_.argsort()[:, ::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features = [feature_names[ind] for ind in top_feature_indexes]

    # top_features_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
    top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features'] = top_features
    cluster_details[cluster_num]['top_features_value'] = top_feature_values
    filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
    filenames = filenames.values.tolist()

    cluster_details[cluster_num]['filenames'] = filenames

  return cluster_details

get_cluster_details()를 호출하면 dictionary를 원소로 가지는 리스트인 cluster_details를 반환한다. 이 cluster_details에는 개별 군집번호, 핵심 단어, 핵심단어 중심 위치 상댓값, 파일명 속성 값 정보가 있는데, 이를 좀 더 보기 좋게 표현하기 위해서 별도의 print_cluster_details() 함수를 만든다.

In [27]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('###### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명:', cluster_detail['filenames'][:7])
    print('==============================================')

앞서 생성한 get_cluster_details(), print_cluster_details()를 호출한다.

In [28]:
feature_names = tfidf_vect.get_feature_names_out() # Use get_feature_names_out() for scikit-learn version 1.0 or newer(코드 오류 수정)

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                    feature_names=feature_names, clusters_num=3, top_n_features=10)

print_cluster_details(cluster_details)

###### Cluster 0
Top features: ['room', 'hotel', 'service', 'interior', 'staff', 'seat', 'location', 'mileage', 'comfortable', 'gas']
Reviews 파일명: ['/content/topics/quality_toyota_camry_2007', '/content/topics/staff_swissotel_chicago', '/content/topics/transmission_toyota_camry_2007', '/content/topics/location_holiday_inn_london', '/content/topics/rooms_bestwestern_hotel_sfo', '/content/topics/location_bestwestern_hotel_sfo', '/content/topics/service_bestwestern_hotel_sfo']
###### Cluster 1
Top features: ['battery', 'screen', 'video', 'battery life', 'life', 'performance', 'size', 'sound', 'ipod', 'camera']
Reviews 파일명: ['/content/topics/screen_netbook_1005ha', '/content/topics/screen_ipod_nano_8gb', '/content/topics/size_asus_netbook_1005ha', '/content/topics/battery-life_amazon_kindle', '/content/topics/battery-life_netbook_1005ha', '/content/topics/battery-life_ipod_nano_8gb', '/content/topics/sound_ipod_nano_8gb']
###### Cluster 2
Top features: ['food', 'screen', 'direction', 'voic

포터블 전자제품 리뷰 군집인 Cluster #0에서는 화면과 배터리 수명 등이 핵심 단어로 군집화되었다.  

자동차 리뷰 군집인 Cluster #1에서는 실내 인테리어, 좌석, 연료 효율 등이 핵심 단어로 군집화되었다. 토요타, 혼다와 같은 일본 자동차의 경우 실내 인테리어와 연료 효율, 편안함 등이 주요 관심사로 보인다.  

호텔 리뷰 군집인 Cluster #2에서는 방, 서비스, 청소 상태, 직원들의 서비스, 위치 등이 핵심 단어로 군집화되어 주요 관심사로 보인다.